In [18]:
player_mode = ["a", "b", "c"]
pawns_mode = [mode for mode in player_mode for _ in range(4)]

pawns_mode

['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'c', 'c', 'c', 'c']

In [10]:
num_player = 4
pawn = [F"p{i}_pawn{j}" for i in range(num_player) for j in range(4)]
pawn

['p0_pawn0',
 'p0_pawn1',
 'p0_pawn2',
 'p0_pawn3',
 'p1_pawn0',
 'p1_pawn1',
 'p1_pawn2',
 'p1_pawn3',
 'p2_pawn0',
 'p2_pawn1',
 'p2_pawn2',
 'p2_pawn3',
 'p3_pawn0',
 'p3_pawn1',
 'p3_pawn2',
 'p3_pawn3']

In [39]:
import pandas as pd
import numpy as np

class SmartPawn:
    aggressive = {}
    kind = {}
    Strategy = {}
    home_zone = {"P0": 46,
                  "P1": 10,
                  "P2": 22,
                  "P3": 34}
    start_zone = {"P0": 0,
                  "P1": 12,
                  "P2": 24,
                  "P3": 36}
    def __init__(self, player_mode):
        self.num_player = len(player_mode)
        pawns = [F"P{i}_pawn{j}" for i in range(self.num_player) for j in range(4)]
        pawns_mode = [mode for mode in player_mode for _ in range(4)]
        initial_loc = ["not in game" for _ in range(0, self.num_player*4)]
        inital_pos = ["not in game" for _ in range(0, self.num_player*4)]
        num_home_zone = [0 for _ in range(0, self.num_player*4)]
        self.main_df = pd.DataFrame({
            "pawn_id": pawns,
            "mode": pawns_mode,
            "loc": initial_loc,
            "position": inital_pos,
            "num_home_zone": num_home_zone
        })
test = SmartPawn(player_mode=["Strategy", "kind", "aggressive"])
test.main_df

,pawn_id,mode,loc,position,num_home_zone
0,P0_pawn0,Strategy,not in game,not in game,0
1,P0_pawn1,Strategy,not in game,not in game,0
2,P0_pawn2,Strategy,not in game,not in game,0
3,P0_pawn3,Strategy,not in game,not in game,0
4,P1_pawn0,kind,not in game,not in game,0
5,P1_pawn1,kind,not in game,not in game,0
6,P1_pawn2,kind,not in game,not in game,0
7,P1_pawn3,kind,not in game,not in game,0
8,P2_pawn0,aggressive,not in game,not in game,0
9,P2_pawn1,aggressive,not in game,not in game,0


In [10]:
import pandas as pd
import numpy as np

class SmartPawn:
    aggressive = {}
    kind = {}
    Strategy = {}
    home_zone = {"P0": 46,
                  "P1": 10,
                  "P2": 22,
                  "P3": 34}
    start_zone = {"P0": 0,
                  "P1": 12,
                  "P2": 24,
                  "P3": 36}
    def __init__(self, player_mode):
        self.num_player = len(player_mode)
        pawns = [F"P{i}_pawn{j}" for i in range(self.num_player) for j in range(4)]
        pawns_mode = [mode for mode in player_mode for _ in range(4)]
        initial_loc = ["not in game" for _ in range(0, self.num_player*4)]
        inital_pos = ["not in game" for _ in range(0, self.num_player*4)]
        num_home_zone = [0 for _ in range(0, self.num_player*4)]
        self.main_df = pd.DataFrame({
            "pawn_id": pawns,
            "mode": pawns_mode,
            "loc": initial_loc,
            "position": inital_pos,
            "num_home_zone": num_home_zone
        })
    
    def move(self, player, rolled_number):
        target_pawns_df = self.main_df[~self.main_df['pawn_id'].str.startswith(F"P{player}_")]
        loc_target_pawns = target_pawns_df["loc"].values

        pawns_profile = pd.DataFrame(
            self.__make_pawns_profile(player, rolled_number, target_pawns_df, loc_target_pawns),
            index=[f"P{player}_pawn{i}" for i in range(4)]
        )        
        print(pawns_profile)
    
    # private methods:

    def __make_pawns_profile(self, player, rolled_number, target_pawns_df, loc_target_pawns):
        can_move = [self.__can_move(F"P{player}_pawn{i}") for i in range(4)]

        can_hit = [self.__can_hit(F"P{player}_pawn{i}", rolled_number, loc_target_pawns) for i in range(4)]

        if rolled_number == 6:
            can_come_in_game = [self.__can_come_in_game(F"P{player}_pawn{i}") for i in range(4)]
        else:
            can_come_in_game = [0 for _ in range(4)]
        
        can_end_round = [self.__can_end_round(F"P{player}_pawn{i}", rolled_number) for i in range(4)]

        return {"can_move": can_move,
                "can_hit": can_hit,
                "can_come_in_game": can_come_in_game,
                "can_end_round": can_end_round}

    def __can_move(self, pawn_id):
        pos_pawn = self.main_df.loc[self.main_df["pawn_id"] == pawn_id, "position"].values[0]
        if pos_pawn == "in game":
            return 1
        else:
            return 0

    def __can_hit(self, pawn_id, rolled_number, loc_target_pawns):
        loc_pawn = self.main_df.loc[self.main_df["pawn_id"] == pawn_id, "loc"].values[0]
        if loc_pawn in ["not in game", "finish game"]:
            return 0
        
        loc_pawn += rolled_number
        if loc_pawn > 47:
            loc_pawn = loc_pawn - 47
            # 47, Number of houses in the game
        
        if loc_pawn in loc_target_pawns:
            return 1
        else:
            return 0
        
    def __can_come_in_game(self, pawn_id):
        pos_pawn = self.main_df.loc[self.main_df["pawn_id"] == pawn_id, "position"].values[0]

        if pos_pawn == "not in game":
            return 1
        else:
            return 0
    
    def __can_end_round(self, pawn_id, rolled_number):
        player = pawn_id[:2]

        pos_pawn = self.main_df.loc[self.main_df['pawn_id'] == pawn_id, "position"].values[0]
        if pos_pawn == "not in game":
            return 0
        
        loc_pawn = self.main_df.loc[self.main_df['pawn_id'] == pawn_id, "loc"].values[0]
        for i in range(1, rolled_number+1):
            loc_pawn += 1
            if loc_pawn == self.home_zone[player]:
                num_home_zone_pawn = (rolled_number - i) + 1
                if num_home_zone_pawn in [1, 2, 3, 4]: 
                    full_home_zone = self.main_df[self.main_df["pawn_id"].str.startswith(player)]["num_home_zone"].values
                    if not num_home_zone_pawn in full_home_zone:
                        return 1
                    else: 
                        return 0
                else:
                    return 0
            else:
                return 0
        return 0
        

test = SmartPawn(player_mode=["Strategy", "kind", "aggressive"])
test.move(0, 6)

          can_move  can_hit  can_come_in_game  can_end_round
P0_pawn0         0        0                 1              0
P0_pawn1         0        0                 1              0
P0_pawn2         0        0                 1              0
P0_pawn3         0        0                 1              0


'P0'